# Install

In [2]:
# ! pip install -U agno
# ! pip install -U  python-dotenv rich
# ! pip install -U openai groq
# ! pip install sqlalchemy
# stop

# Load Secrets

In [4]:
# Import necessary libraries
from dotenv import load_dotenv
import os

# Load the environment variables from the .env file
load_dotenv()

# Access the API keys from the environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
gro_api_key = os.getenv('Groq_API_KEY')

# Print the keys (optional, for testing purposes)
print(f"OpenAI API Key: {bool(openai_api_key)}")
print(f"Gro API Key: {bool(gro_api_key)}")


OpenAI API Key: True
Gro API Key: True


# Question and Answer

In [28]:
import time
qa_data = [
  # {
  #   "question": "Share a childhood memory you have.",
  #   "answer": "I remember playing hide and seek with my friends in the backyard. I always loved being the seeker because it gave me control over the game."
  # },
  # {
  #   "question": "Describe a time when you took a risk.",
  #   "answer": "I once decided to move to a new city alone for a job opportunity, not knowing anyone there. It felt terrifying but it paid off in the end."
  # },
    {
    "question": "リスクを取った時のエピソードを教えてください。",
    "answer": "私は一度、仕事のチャンスで新しい街に一人で引っ越す決断をしました。そこでは誰も知り合いがいなくて、最初はとても不安で怖かったです。新しい環境に馴染むために、何度も自分を励ましながら頑張りましたが、最初の数ヶ月は本当に孤独に感じました。それでも新しい仕事でスキルを身につけ、人間関係も徐々に築いていくことができました。結果として、今ではその決断が大きな成長につながったと感じています。"
    },
  # {
  #   "question": "What motivates you to get out of bed every day?",
  #   "answer": "I get motivated by the thought of new opportunities to learn and grow. Whether it's a new project or an exciting challenge, I thrive on being productive."
  # },
  # {
  #   "question": "How do you handle failure?",
  #   "answer": "I treat failure as a learning opportunity. Instead of feeling discouraged, I analyze what went wrong and try to improve next time."
  # },
  # {
  #   "question": "What is one of your proudest achievements?",
  #   "answer": "Completing a marathon after months of intense training. It taught me the importance of perseverance and setting clear goals."
  # },
  # {
  #   "question": "How would you describe your ideal weekend?",
  #   "answer": "My ideal weekend involves a balance of relaxation and adventure—maybe a hike in the mountains or reading a good book by the beach."
  # },
  # {
  #   "question": "Can you think of a moment when you had to lead a team?",
  #   "answer": "I led a project at work where I had to coordinate a team of five people. It was challenging at first, but by encouraging open communication, we managed to complete the project ahead of schedule."
  # },
  # {
  #   "question": "How do you deal with stressful situations?",
  #   "answer": "I stay calm and try to focus on solving the problem at hand. If needed, I take a short break to reset my mind before tackling the issue."
  # },
  # {
  #   "question": "If you could change one thing about yourself, what would it be?",
  #   "answer": "I would work on being more patient with others, especially in situations that are beyond their control. I tend to be impatient at times."
  # },
  # {
  #   "question": "Describe your perfect day.",
  #   "answer": "A perfect day would start with a peaceful morning walk, followed by a productive workday, and ending with dinner with close friends, laughing and sharing stories."
  # }
]


# OpenAI Agents

In [31]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.models.groq import Groq
from pydantic import BaseModel, Field
import pandas as pd

In [33]:
from pydantic import BaseModel, Field
from typing import Dict

class PersonalityTraits(BaseModel):
    positive: Dict[str, int] = Field(..., description="Positive personality traits with values from 0 to 100")
    negative: Dict[str, int] = Field(..., description="Negative personality traits with values from 0 to 100")
    quote: str = Field(..., description="A motivational or descriptive quote based on the analysis")


In [35]:
# Record start time
start_time = time.time()
# Create an OpenAI agent with PersonalityTraits response model
openai_agent = Agent(
    model=OpenAIChat(id="gpt-4o"),
    response_model=PersonalityTraits,
    markdown=True
)


# Function to generate personality traits using OpenAI
def get_openai_response(question, answer):
    task = f"Question: {question}\nAnswer: {answer}\n\nGenerate the following output: Positive: Confidence: 78%, Bravery: 45%, ...\nNegative: Fear: 12%, Insecurity: 6%\nQuote: 'You are a lone army in a jungle to rule.'"
    response = openai_agent.run(task)
    return response.content

# Collect OpenAI responses in a DataFrame
responses = []
for qa in qa_data:
    openai_response = get_openai_response(qa["question"], qa["answer"])
    responses.append({
        "question": qa["question"],
        "answer": qa["answer"],
        "openai_response": openai_response
    })

# Store responses in a DataFrame
openai_df = pd.DataFrame(responses)


# Record end time
end_time = time.time()
# Calculate elapsed time
elapsed_time = end_time - start_time
print(f"Elapsed Time: {elapsed_time} seconds")

Elapsed Time: 4.26878023147583 seconds


In [36]:
openai_df

,question,answer,openai_response
0,リスクを取った時のエピソードを教えてください。,私は一度、仕事のチャンスで新しい街に一人で引っ越す決断をしました。そこでは誰も知り合いがいな...,"positive={'Confidence': 75, 'Bravery': 60, 'Ad..."


In [39]:
from IPython.display import display, Markdown

# Iterate over the rows to display OpenAI responses in a table format using Markdown
for index, row in openai_df.iterrows():
    # Access the positive and negative traits and format them as key-value pairs
    positive_traits = "\n".join([f"**{k}**: {v}" for k, v in row['openai_response'].positive.items()])
    negative_traits = "\n".join([f"**{k}**: {v}" for k, v in row['openai_response'].negative.items()])
    
    markdown_output = f"""
### **Question:**
**{row['question']}**

---

### **Answer:**
{row['answer']}

---

### **OpenAI Response:**

| Trait            | Value (0-100) |
|------------------|---------------|
"""
    # Add positive traits to the table
    for trait, value in row['openai_response'].positive.items():
        markdown_output += f"| **{trait}** | {value} |\n"
    
    # Add negative traits to the table
    for trait, value in row['openai_response'].negative.items():
        markdown_output += f"| **{trait}** | {value} |\n"

    markdown_output += f"""
---

### **Quote:**
*"{row['openai_response'].quote}"*

---
"""  # Adds nice spacing and clear sections
    display(Markdown(markdown_output))




### **Question:**
**リスクを取った時のエピソードを教えてください。**

---

### **Answer:**
私は一度、仕事のチャンスで新しい街に一人で引っ越す決断をしました。そこでは誰も知り合いがいなくて、最初はとても不安で怖かったです。新しい環境に馴染むために、何度も自分を励ましながら頑張りましたが、最初の数ヶ月は本当に孤独に感じました。それでも新しい仕事でスキルを身につけ、人間関係も徐々に築いていくことができました。結果として、今ではその決断が大きな成長につながったと感じています。

---

### **OpenAI Response:**

| Trait            | Value (0-100) |
|------------------|---------------|
| **Confidence** | 75 |
| **Bravery** | 60 |
| **Adaptability** | 70 |
| **Resilience** | 85 |
| **Fear** | 20 |
| **Insecurity** | 15 |

---

### **Quote:**
*""Taking a leap into the unknown can be the spark that ignites your greatest growth.""*

---


# Groq Agent

In [41]:
# Record start time
start_time = time.time()

# Create a Groq agent with PersonalityTraits response model
groq_agent = Agent(
    model=Groq(id="llama-3.3-70b-versatile"),
    response_model=PersonalityTraits,
    markdown=True
)

# Function to generate personality traits using Groq
def get_groq_response(question, answer):
    task = f"Question: {question}\nAnswer: {answer}\n\nGenerate the following output: Positive: Confidence: 78%, Bravery: 45%, ...\nNegative: Fear: 12%, Insecurity: 6%\nQuote: 'You are a lone army in a jungle to rule.'"
    response = groq_agent.run(task)
    return response.content

# Collect Groq responses in a DataFrame
responses = []
for qa in qa_data:
    groq_response = get_groq_response(qa["question"], qa["answer"])
    responses.append({
        "question": qa["question"],
        "answer": qa["answer"],
        "groq_response": groq_response
    })

# Store responses in a DataFrame
groq_df = pd.DataFrame(responses)
groq_df



# Record end time
end_time = time.time()
# Calculate elapsed time
elapsed_time = end_time - start_time
print(f"Elapsed Time: {elapsed_time} seconds")

Elapsed Time: 2.465466022491455 seconds


In [42]:
# Iterate over the rows to display Groq responses in a table format using Markdown
for index, row in groq_df.iterrows():
    # Access the positive and negative traits and format them as key-value pairs
    positive_traits = "\n".join([f"**{k}**: {v}" for k, v in row['groq_response'].positive.items()])
    negative_traits = "\n".join([f"**{k}**: {v}" for k, v in row['groq_response'].negative.items()])
    
    markdown_output = f"""
### **Question:**
**{row['question']}**

---

### **Answer:**
{row['answer']}

---

### **Groq Response:**

| Trait            | Value (0-100) |
|------------------|---------------|
"""
    # Add positive traits to the table
    for trait, value in row['groq_response'].positive.items():
        markdown_output += f"| **{trait}** | {value} |\n"
    
    # Add negative traits to the table
    for trait, value in row['groq_response'].negative.items():
        markdown_output += f"| **{trait}** | {value} |\n"

    markdown_output += f"""
---

### **Quote:**
*"{row['groq_response'].quote}"*

---
"""  # Adds nice spacing and clear sections
    display(Markdown(markdown_output))



### **Question:**
**リスクを取った時のエピソードを教えてください。**

---

### **Answer:**
私は一度、仕事のチャンスで新しい街に一人で引っ越す決断をしました。そこでは誰も知り合いがいなくて、最初はとても不安で怖かったです。新しい環境に馴染むために、何度も自分を励ましながら頑張りましたが、最初の数ヶ月は本当に孤独に感じました。それでも新しい仕事でスキルを身につけ、人間関係も徐々に築いていくことができました。結果として、今ではその決断が大きな成長につながったと感じています。

---

### **Groq Response:**

| Trait            | Value (0-100) |
|------------------|---------------|
| **Confidence** | 78 |
| **Bravery** | 45 |
| **Resilience** | 90 |
| **Adaptability** | 85 |
| **Fear** | 12 |
| **Insecurity** | 6 |
| **Anxiety** | 8 |

---

### **Quote:**
*"You are a lone army in a jungle to rule."*

---
